<a href="https://colab.research.google.com/github/NakshatraDigala/Prodigy-tasks/blob/main/Prodigy_GPT_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import files

# Upload 'my_poems.txt' that you got from me earlier
uploaded = files.upload()


Saving my_poems.txt to my_poems.txt


In [ ]:
from datasets import Dataset

# Read file
with open("my_poems.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Split into non-empty lines
lines = [line.strip() for line in text.split('\n') if line.strip()]
dataset = Dataset.from_dict({"text": lines})


In [ ]:
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # avoid padding issues

# Updated tokenization function (includes labels)
def tokenize_fn(example):
    tokens = tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()  # 👈 KEY LINE: sets targets
    return tokens

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])


Map:   0%|          | 0/77 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer
import os

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# ✅ Disable Weights & Biases (wandb)
os.environ["WANDB_DISABLED"] = "true"

# ✅ Define training arguments (no evaluation_strategy used)
training_args = TrainingArguments(
    output_dir="./gpt2-poems",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2
)

# ✅ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# ✅ Start training
trainer.train()
trainer.save_model("./gpt2-poems")
tokenizer.save_pretrained("./gpt2-poems")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-15-1041822255.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.791700
100,0.251800


('./gpt2-poems/tokenizer_config.json',
 './gpt2-poems/special_tokens_map.json',
 './gpt2-poems/vocab.json',
 './gpt2-poems/merges.txt',
 './gpt2-poems/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer from saved directory
generator = pipeline(
    "text-generation",
    model="./gpt2-poems",
    tokenizer="./gpt2-poems"
)

# Provide your prompt
prompt = "In the silence of the night,"

# Generate output
output = generator(prompt, max_length=60, do_sample=True, top_k=50)

# Print the result
print("Generated poem:\n")
print(output[0]["generated_text"])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated poem:

In the silence of the night, my eyes remain, for all the love I feel.


In [ ]:
from transformers import pipeline

# Load model and tokenizer
generator = pipeline(
    "text-generation",
    model="./gpt2-poems",
    tokenizer="./gpt2-poems"
)

prompt = "In the silence of the night,"

# ✅ Greedy
print("Greedy:\n", generator(prompt, max_new_tokens=60, do_sample=False)[0]["generated_text"])

# ✅ Top-k Sampling
print("\nTop-k Sampling:\n", generator(prompt, max_new_tokens=60, do_sample=True, top_k=50)[0]["generated_text"])

# ✅ Top-p Sampling
print("\nTop-p Sampling:\n", generator(prompt, max_new_tokens=60, do_sample=True, top_p=0.9)[0]["generated_text"])

# ✅ Beam Search
print("\nBeam Search:\n", generator(prompt, max_new_tokens=60, num_beams=5, early_stopping=True)[0]["generated_text"])


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Greedy:
 In the silence of the night, I feel like a lost child

Top-k Sampling:
 In the silence of the night, it is only the calmest breath in the world

Top-p Sampling:
 In the silence of the night, the light of the moon shone in his eyes

Beam Search:
 In the silence of the night, his eyes were filled with hope.


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📝 Prompt: In the silence of the night,

🔹 Greedy:
 In the silence of the night, I feel like a lost child

🔹 Top-k Sampling:
 In the silence of the night, I wondered what my life would be like if I was all alone

🔹 Top-p Sampling:
 In the silence of the night,


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔹 Beam Search:
 In the silence of the night, I feel like I am the only one in the room


📝 Prompt: Love is the only thing that

🔹 Greedy:
 Love is the only thing that defines me

🔹 Top-k Sampling:
 Love is the only thing that's keeping me safe

🔹 Top-p Sampling:
 Love is the only thing that matters


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔹 Beam Search:
 Love is the only thing that defines me


📝 Prompt: She looked into his eyes and

🔹 Greedy:
 She looked into his eyes and smiled.

🔹 Top-k Sampling:
 She looked into his eyes and then smiled.

🔹 Top-p Sampling:
 She looked into his eyes and smiled.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔹 Beam Search:
 She looked into his eyes and smiled.


📝 Prompt: Sometimes I wonder if

🔹 Greedy:
 Sometimes I wonder if my eyes are the only ones that see me

🔹 Top-k Sampling:
 Sometimes I wonder if I've lost sight of you

🔹 Top-p Sampling:
 Sometimes I wonder if I am the only person who knows me

🔹 Beam Search:
 Sometimes I wonder if I am the only one in the room

